In [1]:
import threading
import time

def count():
    x = 0
    for _ in range(10**7):
        x += 1

# Run two threads
start = time.time()
t1 = threading.Thread(target=count)
t2 = threading.Thread(target=count)

t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print(f"Time taken with two threads: {end - start:.2f} seconds")


Time taken with two threads: 0.35 seconds


In [2]:
start = time.time()
count()
count()
end = time.time()
print(f"Time taken in single thread: {end - start:.2f} seconds")


Time taken in single thread: 0.35 seconds


In [3]:
import multiprocessing
import time

def count():
    x = 0
    for _ in range(10**7):
        x += 1

if __name__ == "__main__":
    start = time.time()
    p1 = multiprocessing.Process(target=count)
    p2 = multiprocessing.Process(target=count)

    p1.start()
    p2.start()
    p1.join()
    p2.join()
    end = time.time()

    print(f"Time taken with multiprocessing: {end - start:.2f} seconds")


Time taken with multiprocessing: 0.05 seconds


Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^AttributeError^^: ^Can't get attribute '

In [ ]:
import threading
import multiprocessing
import time

def count():
    x = 0
    for _ in range(10**7):
        x += 1

# 1️⃣ Single-threaded run
def single_thread_test():
    start = time.time()
    count()
    count()
    end = time.time()
    print(f"Single-threaded: {end - start:.2f} seconds")

# 2️⃣ Multi-threaded run (affected by GIL)
def multi_thread_test():
    start = time.time()
    t1 = threading.Thread(target=count)
    t2 = threading.Thread(target=count)
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    end = time.time()
    print(f"Multi-threaded (GIL): {end - start:.2f} seconds")

# 3️⃣ Multiprocessing run (runs in parallel)
def multi_process_test():
    start = time.time()
    p1 = multiprocessing.Process(target=count)
    p2 = multiprocessing.Process(target=count)
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    end = time.time()
    print(f"Multiprocessing: {end - start:.2f} seconds")


if __name__ == "__main__":
    print("Running benchmark tests...\n")
    single_thread_test()
    multi_thread_test()
    multi_process_test()


In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def count():
    x = 0
    for _ in range(10**7):
        x += 1
    return x

def run_threadpool():
    start = time.time()
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(count) for _ in range(2)]
        for future in futures:
            future.result()
    end = time.time()
    print(f"ThreadPoolExecutor (GIL): {end - start:.2f} seconds")

def run_processpool():
    start = time.time()
    with ProcessPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(count) for _ in range(2)]
        for future in futures:
            future.result()
    end = time.time()
    print(f"ProcessPoolExecutor (parallel): {end - start:.2f} seconds")

if __name__ == "__main__":
    run_threadpool()
    run_processpool()


In [ ]:
import time
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

def count_with_progress(_):
    x = 0
    for _ in tqdm(range(10**7), desc="Counting", leave=False):
        x += 1
    return x

def run_with_progress():
    start = time.time()
    with ProcessPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(count_with_progress, i) for i in range(2)]
        for future in tqdm(futures, desc="Overall Progress"):
            future.result()
    end = time.time()
    print(f"ProcessPoolExecutor with progress bars: {end - start:.2f} seconds")

if __name__ == "__main__":
    run_with_progress()


#### Decorators

In [1]:
import time
import pandas as pd

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        print(f"Starting '{func.__name__}'...")
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Finished '{func.__name__}' in {end_time - start_time:.2f} seconds")
        return result
    return wrapper


In [2]:
@timing_decorator
def clean_data(df):
    df = df.dropna()
    df = df[df['age'] > 18]
    df['income'] = df['income'].apply(lambda x: x * 1.1)
    return df

In [4]:
data = {
    'name': ['Alice', 'Bob', 'Charlie', None],
    'age': [25, 17, 35, 28],
    'income': [50000, 40000, 60000, None]
}

df = pd.DataFrame(data)

cleaned_df = clean_data(df)
print(cleaned_df)


Starting 'clean_data'...
Finished 'clean_data' in 0.01 seconds
      name  age   income
0    Alice   25  55000.0
2  Charlie   35  66000.0


In [5]:
from functools import wraps

def log_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling {func.__name__} with args={args} kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper

@log_decorator
def multiply(a, b):
    return a * b

multiply(3, 4)


[LOG] Calling multiply with args=(3, 4) kwargs={}
[LOG] multiply returned 12


12

In [6]:
import time
from functools import wraps

def retry_decorator(retries=3, delay=1):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"[RETRY {i+1}] {func.__name__} failed: {e}")
                    time.sleep(delay)
            raise Exception(f"{func.__name__} failed after {retries} retries.")
        return wrapper
    return decorator

import random

@retry_decorator(retries=5, delay=0.5)
def unstable_process():
    if random.random() < 0.7:
        raise ValueError("Random failure!")
    return "Success!"

print(unstable_process())


[RETRY 1] unstable_process failed: Random failure!
Success!


In [ ]:
from functools import lru_cache

@timing_decorator
@lru_cache(maxsize=10)
def fibonacci(n):
    print(f"Calculating fibonacci({n})")
    if n in (0, 1):
        return n
    return fibonacci(n-1) + fibonacci(n-2)

fibonacci(10) 


In [12]:
@retry_decorator()
@log_decorator
@timing_decorator
def fetch_and_clean_data():
    # simulate a flaky data read
    if random.random() < 0.2:
        raise ValueError("Temporary read error")
    
    df = pd.DataFrame({
        "age": [23, None, 45, 19],
        "income": [50000, 60000, None, 40000]
    })
    return df.dropna()


In [13]:
fetch_and_clean_data()

[LOG] Calling wrapper with args=() kwargs={}
Starting 'fetch_and_clean_data'...
Finished 'fetch_and_clean_data' in 0.01 seconds
[LOG] wrapper returned     age   income
0  23.0  50000.0
3  19.0  40000.0


,age,income
0,23.0,50000.0
3,19.0,40000.0


In [44]:
def my_decorator(func):
    def wrapper():
        # def wrapper2():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
        # return wrapper2
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()


# ⚡ Step-by-Step Again:
# You define the original say_hello().

# You decorate it with @my_decorator.

# Behind the scenes, Python does:

# python
# Copy
# Edit
# say_hello = my_decorator(say_hello)
# my_decorator(say_hello) runs:

# Inside my_decorator, it defines wrapper().

# Then it does return wrapper — giving back the function wrapper.

# Now say_hello is the wrapper function.

# When you finally do say_hello(), you're actually doing wrapper() — that's when it runs.


Something is happening before the function is called.
Hello!
Something is happening after the function is called.


### generators and yeild

In [14]:
def count_up_to(n):
    count = 1
    while count <= n:
        yield count
        count += 1

counter = count_up_to(5)
for number in counter:
    print(number)


1
2
3
4
5
